In [1]:
import json
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
import random
import torch
from torch.optim import Adam
import dgl
from datetime import datetime

from src import *
from pprint import pprint
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

data_dir = './data_passrefinder-p'

In [2]:
from argparse import ArgumentParser
parser = ArgumentParser()

import argparse
from pprint import pprint

parser = argparse.ArgumentParser(description="Parser for model configuration")

parser.add_argument('--feature_file', type=str, default=f"{data_dir}/data/feature_dict.json", help='feature json file')
parser.add_argument('--reuse_rate_file', type=str, default=f"{data_dir}/data/reuse_rate_dict.json", help='reuse rate json file')
# parser.add_argument('--setting', type=str, required=True, help='graph learning setting: inductive/transductive')
# parser.add_argument('--model_path', type=str, required=True, default="./model/", help='model file path')
parser.add_argument('--agg_type', type=str, default="no_hidden", help='aggregation type')
parser.add_argument('--nclient', type=int, default=10, help='number of clients')
# parser.add_argument('--valid', type=float, default=0.2, help='split ratio of validation set')
# parser.add_argument('--test', type=float, default=0.2, help='split ratio of test set')
parser.add_argument('--dropout', type=float, default=0.5, help='dropout ratio')
parser.add_argument('--random_seed', type=int, default=42, help='random seed for initialization')
parser.add_argument('--relu', type=float, default=0.2, help='ReLU threshold')
parser.add_argument('--reuse_th', type=float, default=0.5, help='threshold for reuse')
parser.add_argument('--train_batch_size', type=int, default=2048, help='batch size for training')
parser.add_argument('--eval_batch_size', type=int, default=2048, help='batch size for evaluation')
parser.add_argument('--embed_size', type=int, default=1024, help='size of the embedding layer')
parser.add_argument('--hidden_size', type=int, default=1024, help='size of the hidden layer')
parser.add_argument('--gnn_depth', type=int, default=2, help='depth of the GNN')
parser.add_argument('--max_lr', type=float, default=0.001, help='maximum learning rate')
parser.add_argument('--warmup', type=float, default=0.02, help='warmup ratio for learning rate')
parser.add_argument('--max_epoch', type=int, default=100, help='maximum number of epochs')
# parser.add_argument('--early_stop', type=int, default=40, help='early stopping epochs')
parser.add_argument('--device', type=int, default=0, help='GPU ID')

args = parser.parse_args(args=[])
pprint(args)

torch.manual_seed(args.random_seed)
torch.cuda.manual_seed(args.random_seed)
dgl.seed(args.random_seed)
np.random.seed(args.random_seed)
random.seed(args.random_seed)

Namespace(feature_file='./data_passrefinder-p/data/feature_dict.json', reuse_rate_file='./data_passrefinder-p/data/reuse_rate_dict.json', agg_type='no_hidden', nclient=10, dropout=0.5, random_seed=42, relu=0.2, reuse_th=0.5, train_batch_size=2048, eval_batch_size=2048, embed_size=1024, hidden_size=1024, gnn_depth=2, max_lr=0.001, warmup=0.02, max_epoch=100, device=0)


In [3]:
with open(args.feature_file, 'r') as f:
    feature_dict = json.load(f)

with open(args.reuse_rate_file, 'r') as f:
    reuse_rate_dict = json.load(f)
    


g = construct_graph(feature_dict, reuse_rate_dict, args.reuse_th)
g_dict, node_set_list, target_eids_dict = graph_split_FL(g, args.nclient, args.random_seed)



# train_loader_list = []
# for g, nodes in zip(g_split['train'], node_split['train']):
#     train_loader_list.append(get_data_loader(g, nodes, args.train_batch_size, args.gnn_depth, suffle=True))
# train_nfeat_list = P.pop_train_node_feature()

# valid_loader = get_data_loader(g_split['valid'], node_split['valid'])
# valid_nfeat = P.pop_node_feature('valid')
# test_loader = get_data_loader(g_split['test'], node_split['test'])
# test_nfeat = P.pop_node_feature('test')


P = PassREfinder_FL(g_dict, node_set_list, target_eids_dict, args)
# P.print_graph()

Constructing graph...
Constructing graph... Done
Splitting graph...
Splitting graph... Done


In [17]:
def evaluate_random(P):
    print('Evaluating...')

    node_dict = {}
    for i, node_set in enumerate(P.node_set_list):
        node_dict.update(dict.fromkeys(node_set.numpy().tolist(), i))

    sample_size = 64
    _reuse_rate_top_test = defaultdict(dict)
    for id1, id2, score in zip(P.g_dict['test'].edges(etype='reuse')[0].tolist(), P.g_dict['test'].edges(etype='reuse')[1].tolist(), P.g_dict['test'].edata['rate'][('website', 'reuse', 'website')].tolist()):
        if node_dict[id1] != node_dict[id2]:
            _reuse_rate_top_test[id1][id2] = score
    
    reuse_rate_top_test = {}
    for id1, id2_list in _reuse_rate_top_test.items():
        if len(id2_list) >= sample_size:
            id2_list = {k: v for k, v in sample(id2_list.items(), sample_size)}
            id2_list = {k: v for k, v in sorted(id2_list.items(), key=lambda x: x[1], reverse=True)}
            reuse_rate_top_test[id1] = id2_list

    with torch.no_grad():
        pair_to_score = defaultdict(dict)
        
        for input_nodes, edge_sub, blocks in tqdm(P.test_loader):
            batch_inputs, batch_labels = load_subtensor(*P.test_nfeat, edge_sub, input_nodes, 'cpu')
            blocks = [block.int().to('cpu') for block in blocks]

            edge_tensor = P.g_dict['test'].find_edges(edge_sub.edata['_ID'][('website', 'reuse', 'website')].detach().cpu().numpy(), etype='reuse')
            edge_list = [(src, dst) for src, dst in zip(edge_tensor[0].tolist(), edge_tensor[1].tolist())]
            
            for edge in edge_list:
                pair_to_score[edge[0]][edge[1]] = random.random()

    result = {}
    result['prec@k'] = evaluate_precision(pair_to_score, reuse_rate_top_test)
    result['nDCG@k'] = evaluate_ndcg(pair_to_score, reuse_rate_top_test)
    return result, pair_to_score

In [18]:
%%time
model_dir = "data_passrefinder-p/output/fl_passrefinder/client_10/2025-03-28 13:04:27"
P.model = load_model(os.path.join(model_dir, 'model.pt'), P.model)
result, pair_to_score = evaluate_random(P)

pprint(result['prec@k'])
pprint(result['nDCG@k'])



# with open(os.path.join(output_dir, 'eval_results.json'), 'w') as f:
#     json.dump(result, f)
# with open(os.path.join(output_dir, 'pair_to_score.json'), 'w') as f:
#     json.dump(pair_to_score, f)

Evaluating...


/tmp/ipykernel_1359817/2970022190.py:17: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  id2_list = {k: v for k, v in sample(id2_list.items(), sample_size)}
100%|██████████| 511/511 [01:23<00:00,  6.11it/s]


{'Precision@1': 0.016654330125832718,
 'Precision@10': 0.1567357512953368,
 'Precision@30': 0.46778929188255614,
 'Precision@5': 0.08031088082901554,
 'Precision@50': 0.7824500370096225}
{'nDCG@1': 0.5856387514090987,
 'nDCG@10': 0.6115201959731528,
 'nDCG@30': 0.6589766376870815,
 'nDCG@5': 0.6005190864736963,
 'nDCG@50': 0.7125844666478005}
CPU times: user 19min 7s, sys: 7.23 s, total: 19min 15s
Wall time: 1min 30s
